## Import libraries

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
import os
from dotenv import load_dotenv

## Load environment variables from the .env file into jupyter notebook

In [2]:
load_dotenv()

client_id = os.getenv('SPOTIPY_CLIENT_ID')
client_secret = os.getenv('SPOTIPY_CLIENT_SECRET')
redirect_uri = os.getenv('SPOTIPY_REDIRECT_URI')

print("Client ID and Client Secret are now loaded without showing them in code.")

Client ID and Client Secret are now loaded without showing them in code.


## Initialize the SpotifyOAuth object with the required credentials and scope for authorization

In [3]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id,
                                               client_secret=client_secret,
                                               redirect_uri=redirect_uri,
                                               scope="playlist-modify-public playlist-modify-private"))

## Function to retrieve tracks from a public playlist and display the tracks

In [4]:
def get_playlist_tracks(playlist_id):
    results = sp.playlist_tracks(playlist_id)
    tracks = results['items']
    
    # If the playlist has more than 100 tracks, fetch the remaining tracks
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    
    # Print out track names and artists
    for idx, item in enumerate(tracks):
        track = item['track']
        print(f"{idx+1}. {track['name']} by {track['artists'][0]['name']}")
    
    return tracks

playlist_id = '5io7Q8iDB7sAMxqyTasntj'
tracks = get_playlist_tracks(playlist_id)

1. I've Never Been There by Yann Tiersen
2. Azul by Little Jesus
3. Under Pressure by Queen
4. White Ferrari by Frank Ocean
5. Te Guardo by Silvana Estrada
6. PIENSO EN TU MIRÁ - Cap.3: Celos by ROSALÍA
7. Savior by Rise Against
8. Did you know that there's a tunnel under Ocean Blvd by Lana Del Rey
9. Mia & Sebastian’s Theme by Justin Hurwitz
10. Use Somebody by Kings of Leon


## Function to retrieve audio features for each track

In [6]:
def get_audio_features(track_ids):
    audio_features = sp.audio_features(track_ids)
    return audio_features

# Initialize an empty list to store track information and features
track_data = []

# Extract track IDs and retrieve audio features
for item in tracks:
    track = item['track']
    track_id = track['id']
    
    # Get audio features for the current track
    features = sp.audio_features(track_id)[0]
    
    # Append track information and audio features
    track_info = {
        'track_id': track['id'],  # Fixed this line
        'track_name': track['name'],
        'artist_name': track['artists'][0]['name'],
        'danceability': features['danceability'],
        'energy': features['energy'],
        'tempo': features['tempo'],
        'loudness': features['loudness'],
        'acousticness': features['acousticness'],
        'valence': features['valence'],
        'instrumentalness': features['instrumentalness'],
        'liveness': features['liveness'],
        'mode': features['mode'],  # Fixed: should use mode feature, not speechiness
        'key': features['key'],  # Fixed: use key feature
        'speechiness': features['speechiness'],  # Fixed: removed duplicate time_signature
        'time_signature': features['time_signature']
    }
    
    track_data.append(track_info)

# Create a DataFrame from the track data
fernandas_playlist = pd.DataFrame(track_data)

# Display the DataFrame
fernandas_playlist

,track_id,track_name,artist_name,danceability,energy,tempo,loudness,acousticness,valence,instrumentalness,liveness,mode,key,speechiness,time_signature
0,743kvRcF2vBfvj8scCxbr8,I've Never Been There,Yann Tiersen,0.636,0.3680,99.939,-8.057,0.93300,0.8110,0.423000,0.3860,1,7,0.0363,4
1,3V4TnrGeVCxB2TLZ2pZQYO,Azul,Little Jesus,0.589,0.6490,147.013,-8.536,0.02740,0.5320,0.008130,0.1550,1,7,0.0589,4
2,7rgZTzcjofQJY5kz9FO3OS,Under Pressure,Queen,0.669,0.5260,113.453,-12.745,0.38700,0.4520,0.000000,0.0879,1,2,0.0522,4
3,2LMkwUfqC6S6s6qDVlEuzV,White Ferrari,Frank Ocean,0.501,0.0958,108.741,-15.605,0.77000,0.2040,0.000000,0.2980,1,0,0.0433,4
4,5fGIyvtxYc7BiijDvqOFGN,Te Guardo,Silvana Estrada,0.557,0.3870,156.343,-9.742,0.89800,0.1930,0.000192,0.0892,0,4,0.0435,4
5,5r8mGKhCDIVWb3AG4hm0l0,PIENSO EN TU MIRÁ - Cap.3: Celos,ROSALÍA,0.647,0.6760,164.510,-6.025,0.09090,0.4100,0.022700,0.0947,1,8,0.0862,3
6,1vcxF91pWs9uNwDROuiCPB,Savior,Rise Against,0.554,0.9320,112.488,-3.424,0.00154,0.4780,0.000139,0.4250,0,5,0.0485,4
7,0HEzuLLojblUOaUSdmJ9gl,Did you know that there's a tunnel under Ocean...,Lana Del Rey,0.441,0.3260,119.999,-9.381,0.75100,0.0526,0.016800,0.1200,1,0,0.0299,4
8,1Vk4yRsz0iBzDiZEoFMQyv,Mia & Sebastian’s Theme,Justin Hurwitz,0.476,0.1920,116.542,-15.753,0.97600,0.1140,0.928000,0.1190,1,9,0.0640,3
9,2TYC9SaRhYypUQzB1ZVTG2,Use Somebody,Kings of Leon,0.276,0.7150,137.028,-5.356,0.00552,0.1730,0.000417,0.2010,1,0,0.0432,4


In [7]:
fernandas_playlist.describe()

,danceability,energy,tempo,loudness,acousticness,valence,instrumentalness,liveness,mode,key,speechiness,time_signature
count,10.000000,10.000000,10.00000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,0.534600,0.486680,127.60560,-9.462400,0.484036,0.341960,0.139938,0.197580,0.800000,4.200000,0.050600,3.800000
std,0.117858,0.258981,22.06963,4.165164,0.421937,0.234952,0.306440,0.127229,0.421637,3.521363,0.016035,0.421637
min,0.276000,0.095800,99.93900,-15.753000,0.001540,0.052600,0.000000,0.087900,0.000000,0.000000,0.029900,3.000000
25%,0.482250,0.336500,112.72925,-11.994250,0.043275,0.178000,0.000152,0.100775,1.000000,0.500000,0.043225,4.000000
50%,0.555500,0.456500,118.27050,-8.958500,0.569000,0.307000,0.004273,0.137500,1.000000,4.500000,0.046000,4.000000
75%,0.624250,0.669250,144.51675,-6.533000,0.866000,0.471500,0.021225,0.273750,1.000000,7.000000,0.057225,4.000000
max,0.669000,0.932000,164.51000,-3.424000,0.976000,0.811000,0.928000,0.425000,1.000000,9.000000,0.086200,4.000000


## Calculate average feature values from 'fernandas_playlist'
## Extract track IDs from the DataFrame

In [8]:
avg_danceability = fernandas_playlist['danceability'].mean()
avg_energy = fernandas_playlist['energy'].mean()
avg_valence = fernandas_playlist['valence'].mean()
avg_acousticness = fernandas_playlist['acousticness'].mean()
avg_instrumentalness = fernandas_playlist['instrumentalness'].mean()
avg_tempo = fernandas_playlist['tempo'].mean()

track_ids = list(fernandas_playlist['track_id'])

## Get recommendations based on the average features

In [9]:
recommendations = sp.recommendations(seed_tracks=track_ids[:5],  # You can use up to 5 seed tracks
                                     limit=25,  # Number of recommended tracks
                                     target_danceability=avg_danceability,
                                     target_energy=avg_energy,
                                     target_valence=avg_valence,
                                     target_acousticness=avg_acousticness,
                                     target_instrumentalness=avg_instrumentalness,
                                     target_tempo=avg_tempo)

## Initialize an empty list for new track data

In [10]:
new_track_data = []
recommended_tracks = []
playlist_id = None  # To hold the ID of the newly created playlist

## Create a new playlist

In [12]:
username = "1272030812" 
playlist_name = "Fernandas Recommended Playlist #2"
description = "A playlist based on average features from Fernanda's top 10 tracks."

In [13]:
playlist = sp.user_playlist_create(user=username, name=playlist_name, public=True, description=description)
playlist_id = playlist['id'] 

## Extract track IDs from recommendations and store additional details

In [15]:
for track in recommendations['tracks']:
    recommended_tracks.append(track['id'])
    
    # Store track details in new_track_data list for the new DataFrame
    new_track_data.append({
        'track_id': track['id'],
        'track_name': track['name'],
        'artist_name': track['artists'][0]['name'],
        'playlist_id': playlist_id  # Add the playlist ID here
    })

## Add recommended tracks to the new playlist

In [16]:
sp.playlist_add_items(playlist_id=playlist['id'], items=recommended_tracks)

{'snapshot_id': 'AAAAAsZNc4wM5cfmBVQ1SPsKpDnAClx2'}

## Output the playlist URL

In [17]:
print(f"Playlist created: {playlist['external_urls']['spotify']}")

Playlist created: https://open.spotify.com/playlist/4AuM3jWDJops0kL13cJ7di


## Function to print added track details given their URIs

In [19]:
def print_added_tracks(track_uris):
    """Prints the names of tracks given their URIs."""
    for uri in track_uris:
        track_details = sp.track(uri)
        print(f"Added Track: {track_details['name']} by {', '.join(artist['name'] for artist in track_details['artists'])}")

## Print the details of the tracks added to the playlist

In [20]:
print("\nTracks added to the Recommended Playlist:")
print_added_tracks(recommended_tracks)


Tracks added to the Recommended Playlist:
Added Track: Here With Me by d4vd
Added Track: Ni una Cosa Ni la Otra by Alex Ferreira, Silvana Estrada
Added Track: Time After Time by Cyndi Lauper
Added Track: Estocolmo by Arawato
Added Track: Quisiera by Flor de Jamaica
Added Track: Evening Party by Yann Tiersen
Added Track: Tired by beabadoobee
Added Track: Eternos by Nuevo Vintage
Added Track: Bailando Lentos by Daniela Spalla, Bandalos Chinos
Added Track: Changes - 2015 Remaster by David Bowie
Added Track: Querer by Chetes
Added Track: American Pie by Don McLean
Added Track: Combustión by Jósean Log
Added Track: Angie - Remastered 2009 by The Rolling Stones
Added Track: Good Old-Fashioned Lover Boy - Remastered 2011 by Queen
Added Track: Carpe Diem by Viniloversus, Tessa Ia
Added Track: Banquet by Yann Tiersen
Added Track: Honey, No Estás by Bratty
Added Track: Patience by Guns N' Roses
Added Track: Here Comes The Sun - Remastered 2009 by The Beatles
Added Track: Wish You Were Here by P

## Create a new DataFrame for the recommended tracks and display the Dataframe

In [21]:
# Create a new DataFrame for the recommended tracks
fernandas_recommendation_playlist_2 = pd.DataFrame(new_track_data)

fernandas_recommendation_playlist_2

,track_id,track_name,artist_name,playlist_id
0,0dS2u2UFd88TIzDDaZDLvS,Here With Me,d4vd,4AuM3jWDJops0kL13cJ7di
1,0cKXRG5rWgbjV92suoEXVR,Ni una Cosa Ni la Otra,Alex Ferreira,4AuM3jWDJops0kL13cJ7di
2,7o9uu2GDtVDr9nsR7ZRN73,Time After Time,Cyndi Lauper,4AuM3jWDJops0kL13cJ7di
3,5GDokyd3xHKIjd5q2TZT19,Estocolmo,Arawato,4AuM3jWDJops0kL13cJ7di
4,58s4Lbsh8MBAFoJ7Aeaex8,Quisiera,Flor de Jamaica,4AuM3jWDJops0kL13cJ7di
5,5VEi0s3uuAGlw25t6EJ6Z5,Evening Party,Yann Tiersen,4AuM3jWDJops0kL13cJ7di
6,6F5mZpEEjhsAW8UEqbIpz1,Tired,beabadoobee,4AuM3jWDJops0kL13cJ7di
7,5iuUW0mqjRDUbiVIJTxMcq,Eternos,Nuevo Vintage,4AuM3jWDJops0kL13cJ7di
8,2GOPoyyXoE0N0nIald3Rj9,Bailando Lentos,Daniela Spalla,4AuM3jWDJops0kL13cJ7di
9,0LrwgdLsFaWh9VXIjBRe8t,Changes - 2015 Remaster,David Bowie,4AuM3jWDJops0kL13cJ7di
